In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship

In [2]:
import numpy as np

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///InPTA.db', echo = True)

In [4]:
Base = declarative_base()

In [5]:
class Member(Base):
    __tablename__ = 'members'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    username = Column(String(45))
    email = Column(String(100))

    def __repr__(self):
        return "<Member(id='%s', name='%s', username='%s')>" % (self.id, self.name, self.username)

In [6]:
def hms_to_rad(raj):
    hh, mm, ss = raj.split(':')
    rad = (float(hh) + float(mm)/60. + float(ss)/3600.)*np.pi/12.
    return rad

def dms_to_rad(decj):
    dd, mm, ss = decj.split(':')
    sign = np.sign(float(dd))
    rad = sign*(sign*float(dd) + float(mm)/60. + float(ss)/3600.)*np.pi/180.
    return rad

In [7]:
class Source(Base):
    __tablename__ = 'sources'
    
    jname = Column(String(45), primary_key=True)
    srctype = Column(String(10))
    raj = Column(Float)
    decj = Column(Float)
    
    def __init__(self, jname, srctype, raj, decj):
        self.jname = jname
        self.srctype = srctype
        self.raj = hms_to_rad(raj)
        self.decj = dms_to_rad(decj)
        
    def __repr__(self):
        return "<Source(JName='%s', Type='%s', RA='%s', DEC='%s')>" % (self.jname, self.srctype, self.raj, self.decj)

In [8]:
class Telescope(Base):
    __tablename__ = 'telescopes'
    
    id = Column(String(10), primary_key=True)
    name = Column(String(45))
    lat = Column(Float)
    long = Column(Float)

    def __repr__(self):
        return "<Telescope('%s')>" % (self.id)

In [9]:
class Proposal(Base):
    __tablename__ = 'proposals'
    
    number = Column(String(10), primary_key=True)
    cycle_number = Column(Integer)
    title = Column(String(500))
    telescope_id = Column(String(10), ForeignKey("telescopes.id")) 
    pi_id = Column(Integer, ForeignKey("members.id")) # The foreign key is given like "tablename.fieldname".
    file = Column(String(500))
    
    telescope = relationship("Telescope")
    PI = relationship("Member")

    def __repr__(self):
        return "<Proposal(number='%s',title='%s')>" % (self.number, self.title)

In [10]:
Base.metadata.create_all(engine)

2020-08-27 11:21:31,011 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-27 11:21:31,013 INFO sqlalchemy.engine.base.Engine ()
2020-08-27 11:21:31,017 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-27 11:21:31,019 INFO sqlalchemy.engine.base.Engine ()
2020-08-27 11:21:31,023 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("members")
2020-08-27 11:21:31,025 INFO sqlalchemy.engine.base.Engine ()
2020-08-27 11:21:31,028 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("sources")
2020-08-27 11:21:31,030 INFO sqlalchemy.engine.base.Engine ()
2020-08-27 11:21:31,032 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("telescopes")
2020-08-27 11:21:31,033 INFO sqlalchemy.engine.base.Engine ()
2020-08-27 11:21:31,035 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("proposals")
2020-08-27 11:21:31,037 INFO sqlalchemy.engine.base.Engine ()


In [11]:
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
session.rollback()

In [13]:
#testsrc = Source(jname="J0002+6216", raj="00:02:58.17", decj="+62:16:09.4", srctype="PSR")
#session.add(testsrc)
#session.commit()

testsrc = session.query(Source).filter(Source.jname=='J0002+6216').one()

2020-08-27 11:21:34,260 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-27 11:21:34,274 INFO sqlalchemy.engine.base.Engine SELECT sources.jname AS sources_jname, sources.srctype AS sources_srctype, sources.raj AS sources_raj, sources.decj AS sources_decj 
FROM sources 
WHERE sources.jname = ?
2020-08-27 11:21:34,279 INFO sqlalchemy.engine.base.Engine ('J0002+6216',)


In [14]:
testsrc

<Source(JName='J0002+6216', Type='PSR', RA='0.0064784440172464515', DEC='1.0868039200611601')>

In [17]:
#gmrt = Telescope(id='GMRT', name='Giant Meterwave Radio Telescope', lat=0.33329486135, long=1.29241146296)
#session.add(gmrt)
#session.commit()

gmrt = session.query(Telescope).filter(Telescope.id=='GMRT').one()

2020-08-27 11:24:34,926 INFO sqlalchemy.engine.base.Engine SELECT telescopes.id AS telescopes_id, telescopes.name AS telescopes_name, telescopes.lat AS telescopes_lat, telescopes.long AS telescopes_long 
FROM telescopes 
WHERE telescopes.id = ?
2020-08-27 11:24:34,934 INFO sqlalchemy.engine.base.Engine ('GMRT',)


In [18]:
#testmember = Member(name="Test User", username="testuser", email="testusersassd@gmail.com")
#session.add(testmember)
#session.commit()
testuser = session.query(Member).filter(Member.username=='testuser').one()

2020-08-27 11:24:37,468 INFO sqlalchemy.engine.base.Engine SELECT members.id AS members_id, members.name AS members_name, members.username AS members_username, members.email AS members_email 
FROM members 
WHERE members.username = ?
2020-08-27 11:24:37,480 INFO sqlalchemy.engine.base.Engine ('testuser',)


In [19]:
## This is how you create an object with foreign keys.
testproposal = Proposal(number='37_066', cycle_number=37, title="Bla bla", telescope=gmrt, PI=testuser)

In [22]:
session.add(testproposal)
session.commit()

2020-08-27 11:25:24,946 INFO sqlalchemy.engine.base.Engine INSERT INTO proposals (number, cycle_number, title, "telescope_ID", pi_id, file) VALUES (?, ?, ?, ?, ?, ?)
2020-08-27 11:25:24,953 INFO sqlalchemy.engine.base.Engine ('37_066', 37, 'Bla bla', 'GMRT', 1, None)
2020-08-27 11:25:24,963 INFO sqlalchemy.engine.base.Engine COMMIT
